# Import Packages

In [1]:
import os
import pandas as pd
from pathlib import Path
os.chdir(Path(os.getcwd()).parent.parent)

In [2]:
os.getcwd()

'D:\\LFProjects\\NewPythonProject'

# Load Data

In [3]:
data_long = pd.read_pickle("D:/LFProjects/NewPythonProject/data/trade_hold/原始数据/trade_hold_long.pkl")
data_short = pd.read_pickle("D:/LFProjects/NewPythonProject/data/trade_hold/原始数据/trade_hold_short.pkl")
data_volume = pd.read_pickle("D:/LFProjects/NewPythonProject/data/trade_hold/原始数据/trade_hold_volume.pkl")

## Preprocess Data
1.Add underlying_symbol。\
2.change the column trading_date into datetime。

In [4]:
data_long['underlying_symbol'] = data_long['commodity_id'].str.extract("([A-Z]*)")
data_short['underlying_symbol'] = data_short['commodity_id'].str.extract("([A-Z]*)")
data_volume['underlying_symbol'] = data_volume['commodity_id'].str.extract("([A-Z]*)")

In [5]:
data_long.rename(columns={'trading_date': 'datetime'}, inplace=True)
data_short.rename(columns={'trading_date': 'datetime'}, inplace=True)
data_volume.rename(columns={"trading_date": 'datetime'}, inplace=True)

In [6]:
data_long.head()

,datetime,volume,volume_change,member_name,rank,commodity_id,rank_by,exchange,underlying_symbol
5434387,2009-07-01,175,1,路易达孚,1,A0907,long,DCE,A
5434388,2009-07-01,47,-13,北京中期,2,A0907,long,DCE,A
5434389,2009-07-01,43,2,中晟期货,3,A0907,long,DCE,A
5434390,2009-07-01,6,1,渤海期货,4,A0907,long,DCE,A
5434391,2009-07-02,175,0,路易达孚,1,A0907,long,DCE,A


# 测试数据

通过验证数据中是否同时存在：\
1.以合约为单位的数据（commodity_id字段为合约）\
2.以品种为单位的数据（commodity_id字段为品种）\
导致在聚合时数据被重复相加，导致部分品种的成员持仓总和大于open interest。

方法：检测每个交易日每个品种commodity_id=underlying_symbol的行数占总行数的比例，既以品种为单位的数据的占比。

In [7]:
data_long['tag'] = data_long['commodity_id'] == data_long['underlying_symbol']
check_long_df = data_long.copy()
check_long_df['pct'] = data_long.groupby(['datetime', 'underlying_symbol', 'exchange'])['tag'].transform(lambda x: x.sum()/len(x))

In [8]:
data_short['tag'] = data_short['commodity_id'] == data_short['underlying_symbol']
check_short_df = data_short.copy()
check_short_df['pct'] = data_short.groupby(['datetime', 'underlying_symbol', 'exchange'])['tag'].transform(lambda x: x.sum()/len(x))

In [9]:
data_volume['tag'] = data_volume['commodity_id'] == data_volume['underlying_symbol']
check_volume_df = data_volume.copy()
check_volume_df['pct'] = data_volume.groupby(['datetime', 'underlying_symbol', 'exchange'])['tag'].transform(lambda x: x.sum()/len(x))

## 验证是否异常数据全部来自某一个交易所

In [10]:
check_long_df[(check_long_df['pct'] > 0.0) & (check_long_df['pct'] < 1.0)].exchange.value_counts()

CZCE    1824341
Name: exchange, dtype: int64

In [11]:
check_short_df[(check_short_df['pct'] > 0.0) & (check_short_df['pct'] < 1.0)].exchange.value_counts()

CZCE    1801912
Name: exchange, dtype: int64

In [12]:
check_volume_df[(check_volume_df['pct'] > 0.0) & (check_volume_df['pct'] < 1.0)].exchange.value_counts()

CZCE    1746128
Name: exchange, dtype: int64

结论：异常数据确实全部来自郑商所的数据。

# 解决方案
保留（以单日单品种数据为单位）：\
1.所有数据以合约为单位。\
2.所有数据以品种为单位。\
3.对于同时存在以品种为单位和以合约为单位的数据，仅保留以品种为单位的数据。

In [13]:
check_long_df = check_long_df[(check_long_df['pct']==0.0)|(check_long_df['pct']==1.0)|((check_long_df['pct']>0.0)&(check_long_df['pct']<1.0)&(check_long_df['tag']))]
check_short_df = check_short_df[(check_short_df['pct']==0.0)|(check_short_df['pct']==1.0)|((check_short_df['pct']>0.0)&(check_short_df['pct']<1.0)&(check_short_df['tag']))]
check_volume_df = check_volume_df[(check_volume_df['pct']==0.0)|(check_volume_df['pct']==1.0)|((check_volume_df['pct']>0.0)&(check_volume_df['pct']<1.0)&(check_volume_df['tag']))]

# 检查数据

检查数据的方式检查是否存在聚合每日成交持仓数据会不会大于品种当日的open_interest

In [14]:
from data_manager.DailyDataManager import DailyDataManager
daily_data = DailyDataManager().get_daily_data()

In [15]:
daily_data.head()

,contract,datetime,settlement,upper_limit,high,lower_limit,open,open_interest,volume,low,turnover,close,prev_settlement,underlying_symbol
0,A0901,2009-01-05,3696.0,3920.0,3736.0,3478.0,3736.0,2956.0,248.0,3560.0,9168100.0,3660.0,3699.0,A
1,A0901,2009-01-06,3586.0,3917.0,3679.0,3475.0,3583.0,2956.0,72.0,3580.0,2582200.0,3679.0,3696.0,A
2,A0901,2009-01-07,3599.0,3801.0,3600.0,3371.0,3600.0,2956.0,248.0,3599.0,8926300.0,3599.0,3586.0,A
3,A0901,2009-01-08,3599.0,3814.0,3600.0,3384.0,3599.0,2956.0,72.0,3599.0,2591900.0,3600.0,3599.0,A
4,A0901,2009-01-09,3599.0,3814.0,3600.0,3384.0,3600.0,2806.0,44.0,3599.0,1584000.0,3600.0,3599.0,A


In [30]:
open_interest = daily_data.groupby(['datetime', 'underlying_symbol'])['open_interest'].sum()

In [17]:
volume = daily_data.groupby(['datetime', 'underlying_symbol'])['volume'].sum()

In [18]:
(check_long_df.groupby(by=['datetime', 'underlying_symbol'], as_index=True)['volume'].sum()/open_interest).sort_values(ascending=False)

datetime    underlying_symbol
2020-06-09  LR                   1.0
2021-01-12  RS                   1.0
2020-04-09  LR                   1.0
2020-11-24  RS                   1.0
2020-02-05  WH                   1.0
                                ... 
2021-02-26  WH                   NaN
            WR                   NaN
            Y                    NaN
            ZC                   NaN
            ZN                   NaN
Length: 129218, dtype: float64

In [19]:
(check_short_df.groupby(by=['datetime', 'underlying_symbol'], as_index=True)['volume'].sum()/open_interest).sort_values(ascending=False)

datetime    underlying_symbol
2020-04-09  LR                   1.0
2020-10-16  RS                   1.0
2020-03-27  WH                   1.0
2020-08-21  LR                   1.0
2020-10-16  WH                   1.0
                                ... 
2021-02-26  WH                   NaN
            WR                   NaN
            Y                    NaN
            ZC                   NaN
            ZN                   NaN
Length: 129218, dtype: float64

结论：没有数据大于1.0，说明数据清洗完毕。

# 保存数据

In [20]:
check_long_df.to_pickle("D:/LFProjects/NewPythonProject/data/trade_hold/trade_hold_long.pkl")
check_short_df.to_pickle("D:/LFProjects/NewPythonProject/data/trade_hold/trade_hold_short.pkl")
check_volume_df.to_pickle("D:/LFProjects/NewPythonProject/data/trade_hold/trade_hold_volume.pkl")

In [21]:
data_long.shape

(6459603, 10)

In [22]:
check_long_df.shape

(5185919, 11)

In [32]:
from data_manager.TradeHoldDataManager import TradeHoldDataManager
self = TradeHoldDataManager()
a = self.get_trade_hold_data_by_rank(rank_by='long',group_by_symbol=False)
a.shape

(5185919, 11)

In [33]:
a

,datetime,volume,volume_change,member_name,rank,commodity_id,rank_by,exchange,underlying_symbol,tag,pct
5434387,2009-07-01,175,1,路易达孚,1,A0907,long,DCE,A,False,0.0
5434388,2009-07-01,47,-13,北京中期,2,A0907,long,DCE,A,False,0.0
5434389,2009-07-01,43,2,中晟期货,3,A0907,long,DCE,A,False,0.0
5434390,2009-07-01,6,1,渤海期货,4,A0907,long,DCE,A,False,0.0
5434391,2009-07-02,175,0,路易达孚,1,A0907,long,DCE,A,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1343633,2020-12-31,1816,554,五矿经易期货,16,PF,long,CZCE,PF,True,0.5
1343634,2020-12-31,1688,146,浙商期货,17,PF,long,CZCE,PF,True,0.5
1343635,2020-12-31,1616,276,中信建投,18,PF,long,CZCE,PF,True,0.5
1343636,2020-12-31,1594,1121,国联期货,19,PF,long,CZCE,PF,True,0.5


In [34]:
long = a.groupby(by=['datetime', 'underlying_symbol'], as_index=True)['volume'].sum()

In [35]:
long

datetime    underlying_symbol
2009-07-01  A                    111043
            C                    124448
            CF                    14108
            ER                     6408
            L                     39407
                                  ...  
2021-02-19  UR                    77694
            V                    263061
            Y                    440056
            ZC                   207139
            ZN                   105556
Name: volume, Length: 106879, dtype: int64

In [36]:
open_interest

datetime    underlying_symbol
2009-01-05  A                    393642.0
            AL                   238578.0
            AU                    51320.0
            B                       162.0
            C                    306878.0
                                   ...   
2021-02-26  WH                      865.0
            WR                       70.0
            Y                    701225.0
            ZC                   344314.0
            ZN                   143666.0
Name: open_interest, Length: 125954, dtype: float64

In [37]:
common_index = long.index.intersection(open_interest.index)
long = long.loc[common_index]
open_interest = open_interest.loc[common_index]

In [39]:
factor = (long/open_interest).unstack(level=-1)

In [40]:
factor.max().max()

1.0